<a href="https://colab.research.google.com/github/bahadirbesirkestane/Staj/blob/main/Threading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Threading**
**Thread :**  Bir thread, bir işlem içinde bağımsız olarak çalışabilen bir küçük iş parçasıdır. Python'da threading modülü, birden fazla iş parçacığını yönetmek için kullanılır. Thread içindeki kodlar sırayla işleme girerler, bir önceki kod satırı çalıştırılmadan bir sonraki çalıştırılmaz.



**Multi-Thread :** Birden fazla iş parçacığının bulunduğu senaryodur. Çalışacak olan bir kod bloğunun yanında aynı anda ona paralel olarak çalışması gereken durumlarda kullanılır.


Oluşturulan bir thread'i başlatmak için start() methodunu kullanılır. Thread'i sonlandırmak için join() methodunu kullanılır.

In [7]:
from threading import Thread

def thread_function():
  for i in range(5):
    print("Thread ile Çağrıldı: " + str(i))
    time.sleep(1)
def function():
  for i in range(5):
    print(i)
    time.sleep(3)


In [8]:
thread_fun = Thread(target = thread_function)
thread_fun.start()
function()

Thread ile Çağrıldı: 0
0
Thread ile Çağrıldı: 1
Thread ile Çağrıldı: 2
Thread ile Çağrıldı: 3
1
Thread ile Çağrıldı: 4
2
3
4


>Normal fonksiyon 3 saniye bekleyerek ekrana yazarken Thread ile çağırılan fonksiyon da onunla paralel işlem yaparak, onu engellemeden 1 saniye bekleyerek ekrana yazıyor.

In [2]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

**Thread Senkronizasyonu :** Birden fazla thread aynı verilere veya kaynaklara erişiyorsa, senkronizasyon gereklidir. Bu, veri bozulmalarını veya uyumsuz davranışları önler.

### Lock Kullanarak Senkronizasyon

In [12]:
# Lock Kullanarak Senkronizasyon

import threading

# Paylaşılan kaynak
shared_resource = 0

# Lock nesnesi oluştur
lock = threading.Lock()

def increment_shared_resource():
    global shared_resource
    for _ in range(100000):
        # Lock ile kritik bölgeyi koru
        with lock:
            shared_resource += 1

def decrement_shared_resource():
    global shared_resource
    for _ in range(100000):
        # Lock ile kritik bölgeyi koru
        with lock:
            shared_resource -= 1

# İki thread'i başlat
thread1 = threading.Thread(target=increment_shared_resource)
thread2 = threading.Thread(target=decrement_shared_resource)

thread1.start()
thread2.start()

thread1.join()
thread2.join()

print("Sonuç:", shared_resource)

Sonuç: 0


Bu örnekte, iki thread aynı shared_resource değişkenine erişiyor ve with lock blokları içinde bu değişkeni güvence altına alıyor. Bu sayede veri bozulmalarını önler.

### Semaphore Kullanarak Senkronizasyon

In [16]:
# Semaphore Kullanarak Senkronizasyon
import threading

# Maksimum iki thread'e izin veren bir semafor oluştur
semaphore = threading.Semaphore(2)

def access_shared_resource():
    with semaphore:
        print("Thread erişti.")
        # Kritik bölgey
        print("Thread işini bitirdi.")

# Thread leri başlat
threads = []
for _ in range(5):
    thread = threading.Thread(target=access_shared_resource)
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

print("İşlem tamamlandı.")

Thread erişti.
Thread işini bitirdi.
Thread erişti.
Thread işini bitirdi.
Thread erişti.
Thread işini bitirdi.
Thread erişti.
Thread işini bitirdi.
Thread erişti.
Thread işini bitirdi.
İşlem tamamlandı.


Bu örnekte, bir semafor oluşturulur ve her thread, semaforun izin verdiği kadar erişim hakkına sahip olmak için semaforu kullanır. Bu, aynı anda yalnızca belirli sayıda thread'in belirli bir kritik bölgeye erişmesine izin verir.

**BoundedSemaphore()**

In [5]:
import threading


# BoundedSemaphore kullanımı
bounded_semaphore = threading.BoundedSemaphore(2)  # Aynı anda en fazla iki thread erişebilir

def access_resource_bounded(client_id):
    bounded_semaphore.acquire()
    print(f"Client {client_id} kaynağı aldı.")
    # Kaynağa erişim süresi simüle ediliyor
    bounded_semaphore.release()
    print(f"Client {client_id} kaynağı bıraktı.")

# 4 farklı client'i temsil eden thread'leri başlat
threads_bounded = []
for i in range(4):
    thread = threading.Thread(target=access_resource_bounded, args=(i,))
    threads_bounded.append(thread)
    thread.start()

# Thread'lerin bitmesini bekle
for thread in threads_bounded:
    thread.join()

print("BoundedSemaphore ile işlem tamamlandı.")


Client 0 kaynağı aldı.
Client 0 kaynağı bıraktı.
Client 1 kaynağı aldı.
Client 1 kaynağı bıraktı.
Client 2 kaynağı aldı.
Client 2 kaynağı bıraktı.
Client 3 kaynağı aldı.
Client 3 kaynağı bıraktı.
BoundedSemaphore ile işlem tamamlandı.


 BoundedSemaphore, Semaphore'un sınırlanmış bir versiyonudur. Bu, belirli bir sınırlamayı zaten içerir ve daha fazla izin vermez. Yani, BoundedSemaphore oluşturulduğunda, başlangıçta belirli bir sayıda izin verilir ve bu sayıdan fazla izin verilmez.

### Condition Kullanarak Senkronizasyon

In [19]:
# Condition Kullanarak Senkronizasyon
import threading

# Condition nesnesi oluştur
condition = threading.Condition()

# Paylaşılan kaynak
shared_resource = []

def producer():
    for i in range(5):
        with condition:
            shared_resource.append(i)
            print(f"Üretici üretti: {i}")
            condition.notify()  # Bekleyen bir tüketiciyi uyandır

def consumer():
    for i in range(5):
        with condition:
            while not shared_resource:
                condition.wait()  # Üreticiyi beklemeye al
            item = shared_resource.pop(0)
            print(f"Tüketici tüketti: {item}")

# Üretici ve tüketici thread'lerini başlat
producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

producer_thread.start()
consumer_thread.start()

producer_thread.join()
consumer_thread.join()

Üretici üretti: 0
Üretici üretti: 1
Üretici üretti: 2
Üretici üretti: 3
Üretici üretti: 4
Tüketici tüketti: 0
Tüketici tüketti: 1
Tüketici tüketti: 2
Tüketici tüketti: 3
Tüketici tüketti: 4


Condition sınıfı, bir veya daha fazla thread'in bir koşulu beklemesini ve diğer bir thread'in bu koşulu karşılayarak diğer threadleri uyandırmasını sağlar. Bu örnekte, Condition kullanılarak üretici ve tüketici thread'lerinin paylaşılan kaynağa erişimi senkronize edilir.

### Event Kullanarak Senkronizasyon

In [20]:
# Event Kullanarak Senkronizasyon
import threading

# Event nesnesi oluştur
event = threading.Event()

def waiter():
    print("Bekleme başladı.")
    event.wait()  # Event'in tetiklenmesini bekle
    print("Bekleme sona erdi.")

def setter():
    print("Event tetiklendi.")
    event.set()  # Event'i tetikle

# Bekleyen ve tetikleyen thread'leri başlat
waiter_thread = threading.Thread(target=waiter)
setter_thread = threading.Thread(target=setter)

waiter_thread.start()
setter_thread.start()

waiter_thread.join()
setter_thread.join()

Bekleme başladı.
Event tetiklendi.
Bekleme sona erdi.


Event sınıfı, bir thread grubunun bir olayın gerçekleşmesini beklemesini ve diğer bir thread'in bu olayı tetiklemesini sağlar.

### Barrier Kullanarak Senkronizasyon

In [6]:
import threading

# Barrier oluştur (4 thread bekleniyor)
barrier = threading.Barrier(4)

def worker(worker_id):
    print(f"Worker {worker_id} başladı.")
    # İşçi işlemi burada gerçekleştirir
    print(f"Worker {worker_id} işlemi tamamlandı.")
    # Diğer işçileri bekler
    barrier.wait()
    print(f"Worker {worker_id} barrier geçti ve devam ediyor.")

# 4 işçi thread'i oluştur
threads = []
for i in range(4):
    thread = threading.Thread(target=worker, args=(i,))
    threads.append(thread)
    thread.start()

# Thread'lerin bitmesini bekle
for thread in threads:
    thread.join()

print("Tüm işlemler tamamlandı.")


Worker 0 başladı.Worker 1 başladı.
Worker 2 başladı.
Worker 2 işlemi tamamlandı.
Worker 1 işlemi tamamlandı.

Worker 0 işlemi tamamlandı.
Worker 3 başladı.
Worker 3 işlemi tamamlandı.
Worker 3 barrier geçti ve devam ediyor.Worker 2 barrier geçti ve devam ediyor.Worker 1 barrier geçti ve devam ediyor.

Worker 0 barrier geçti ve devam ediyor.

Tüm işlemler tamamlandı.


Barrier sınıfı, n sayısındaki thread'in belirli bir noktada toplanmasını bekler. n thread, barrier'a ulaştığında, hepsi bir araya gelene kadar beklenir ve ardından hepsi birlikte devam eder.

**Global Interpreter Lock (GIL) :** CPython adı verilen Python yorumlayıcısının sadece bir thread'in aynı anda Python kodu çalıştırmasına izin veren bir kilit mekanizmasıdır.

In [21]:
import threading

# Paylaşılan bir değişken
shared_counter = 0

def increment_counter():
    global shared_counter
    for _ in range(1000000):
        shared_counter += 1

# İki thread oluştur
thread1 = threading.Thread(target=increment_counter)
thread2 = threading.Thread(target=increment_counter)

# Thread'leri başlat
thread1.start()
thread2.start()

# Thread'lerin bitmesini bekle
thread1.join()
thread2.join()

# Sonuç
print("Paylaşılan Değer (shared_counter):", shared_counter)


Paylaşılan Değer (shared_counter): 2000000


**Daemon threadler :** Python'da bir ana programın çalışmasını etkilemeden arka planda çalışan ve ana program sona erdiğinde otomatik olarak sona eren threadlerdir. Yani, daemon threadlerin çalışması, ana programın çalışmasını beklemeyecektir. Daemon threadler, genellikle arka planda sürekli olarak çalışan görevleri gerçekleştirmek için kullanılır.

Ana program sona erdiğinde, daemon threadler otomatik olarak sonlandırılır. Ana programın işlevselliğini engellemeden arka planda çalışır. Daemon threadler, genellikle normal (non-daemon) threadlere göre daha düşük önceliğe sahiptir.

In [22]:
import threading
import time

def daemon_function():
    while True:
        print("Daemon thread çalışıyor...")
        time.sleep(1)

# Daemon thread oluştur
daemon_thread = threading.Thread(target=daemon_function)
daemon_thread.setDaemon(True)  # Thread'i daemon olarak işaretle
daemon_thread.start()

# Ana programın çalışması
time.sleep(5)
print("Ana program sona eriyor.")

<ipython-input-22-f8795d4e621f>:11: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  daemon_thread.setDaemon(True)  # Thread'i daemon olarak işaretle


Daemon thread çalışıyor...
Daemon thread çalışıyor...
Daemon thread çalışıyor...
Daemon thread çalışıyor...
Daemon thread çalışıyor...
Ana program sona eriyor.
Daemon thread çalışıyor...


Bu örnekte, daemon_function adlı bir daemon thread oluşturulur ve setDaemon(True) ile daemon olarak işaretlenir. Ana program 5 saniye boyunca çalışırken, daemon thread arka planda çalışmaya devam eder ve ana program sona erdiğinde daemon thread de otomatik olarak sona erer

**ThreadPool (İş Parçacığı Havuzu) :** Çoklu iş parçacığı programlamasında yaygın olarak kullanılan bir tasarım desenidir. Bu tasarım deseni, sınırlı bir iş parçacığı havuzu içinde birden fazla iş parçacığının yeniden kullanılmasını sağlar. Böylece çok sayıda işlemi aynı anda yürütebilir ve kaynak kullanımını optimize edebilirsiniz.

İşlem başlatma maliyetini azaltır ve işlemci kaynaklarını daha etkili bir şekilde kullanmanıza olanak tanır.

In [1]:
import concurrent.futures
import time

# Faktoriyel Hesapla
def calculate_factorial(num):
    result = 1
    for i in range(1, num + 1):
        result *= i
    return result

# ThreadPool oluştur
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # Hesaplanacak faktöriyel sayıları
    sayilar = [5, 10, 15, 20]

    # İş parçacığı havuzunda faktöriyel hesaplaması için görevleri gönder
    futures = {executor.submit(calculate_factorial, num): num for num in sayilar}

    # Sonuçları al
    for future in concurrent.futures.as_completed(futures):
        num = futures[future]
        try:
            result = future.result()
            print(f"{num}! = {result}")
        except Exception as e:
            print(f"Hata: {e}")


15! = 1307674368000
10! = 3628800
5! = 120
20! = 2432902008176640000


**RLock :**Lock ile RLock arasındaki en belirgin fark, Lock’ın kilidini bir başka iş parçacığı açabilir olması, oysa RLock’ın kilidini, kilidi edinmiş olan iş parçacığının açması gerekir.

In [1]:
import threading


class Thread(threading.Thread): # Thread() sınıfını miras alan sınıf
    def __init__(self, lock):
        threading.Thread.__init__(self)
        self.lock = lock

    def run(self):
        self.lock.acquire(blocking=True, timeout=3)
        print("{} çalışıyor.".format(self.name))
        self.lock.acquire(blocking=True, timeout=1) #Bir iş parçacığı RLock kilidini ikinci kez-
        print("{} çalışması bitti.".format(self.name)) # kendi işlemlerini engellemeden elde edebilir.


__lock__ = threading.RLock() # Kilidin oluşturulması
t1 = Thread(lock=__lock__) # Threadlerin oluşturulması
t2 = Thread(lock=__lock__)
t1.start()
t2.start()

Thread-10 çalışıyor.
Thread-10 çalışması bitti.
Thread-11 çalışıyor.
Thread-11 çalışması bitti.


Bu örnekte RLock kilidine sahip olan iş parçacığı t1’dir. Dolayısıyla kilidi sadece o açabilir. Bu örneği çalıştırdığınızda, t1 iş parçacığının kilit edindiğini ama serbest bırakmadığını görüyoruz. Eğer t1 bu kilidi serbest bıraksaydı, iş parçacıkları arasında bekleme süresi olmayacaktı.